In [1]:
!conda env list

# conda environments:
#
                         /lustre/isaac/scratch/ahoust17/aidml
                         /lustre/isaac/scratch/ahoust17/ase_env
                         /lustre/isaac/scratch/ahoust17/gpaw
                         /lustre/isaac/scratch/ahoust17/myapps/conda_env/abtem_test
BO_env                   /nfs/home/ahoust17/.conda/envs/BO_env
abtem                    /nfs/home/ahoust17/.conda/envs/abtem
gpaw_env                 /nfs/home/ahoust17/.conda/envs/gpaw_env
pyTEMlib                 /nfs/home/ahoust17/.conda/envs/pyTEMlib
torch_env                /nfs/home/ahoust17/.conda/envs/torch_env
                         /nfs/home/ahoust17/conda_envs/abtem_env
base                     /sw/isaac/applications/anaconda3/2023.09/rhel8_cascadelake_binary/anaconda3-2023.09



In [1]:
import abtem
import ase
from ase.io import read
from ase.visualize import view
import matplotlib.pyplot as plt
import numpy as np
%matplotlib ipympl


import numpy as np
from ase.neighborlist import NeighborList

from ase import Atoms
from ase.build import bulk
from scipy.ndimage import rotate

In [2]:
def read_and_translate_new_format(input_file, output_file):
    print('Reading')
    with open(input_file, 'r') as f:
        lines = f.readlines()

    print(f"Read {len(lines)} lines from the input file.")
    print(f"First 10 lines: {lines[:10]}")
    # Parse lattice constants
    a, b, c = None, None, None
    for line in lines:
        if '_cell_length_a' in line:
            a = float(line.split()[1])
        elif '_cell_length_b' in line:
            b = float(line.split()[1])
        elif '_cell_length_c' in line:
            c = float(line.split()[1])

    print(a, b, c)
    if None in (a, b, c):
        raise ValueError("Cell lengths a, b, or c not found in the input file.")
    
    print(f"Lattice parameters: a={a}, b={b}, c={c}")

    # Find the atom site data
    atom_data_start = None
    for i, line in enumerate(lines):
        if '_atom_site_fract_z' in line:  # Identify the line after the header
            atom_data_start = i + 1
            break
    
    if atom_data_start is None:
        raise ValueError("Atom site data not found in the input file.")
    
    print(f"Atom site data starts at line {atom_data_start}.")

    # Extract atom site data
    atom_sites = []
    for line in lines[atom_data_start:]:
        parts = line.split()
        if len(parts) < 5:  # Stop if the loop_ section ends or there are fewer columns
            break
        try:
            _, element, x, y, z = parts
            atom_sites.append(('Fe', float(x), float(y), float(z)))
        except ValueError as e:
            print(f"Error parsing line: {line}")
            continue
    
    if not atom_sites:
        raise ValueError("No atom site data extracted from the file.")

    print(f"Extracted {len(atom_sites)} atom sites.")

    # Convert fractional coordinates to Cartesian coordinates
    converted_atoms = []
    for element, x, y, z in atom_sites:
        x_cartesian = x * a
        y_cartesian = y * b
        z_cartesian = z * c
        converted_atoms.append((element, x_cartesian, y_cartesian, z_cartesian))
    
    print(f"Converted {len(converted_atoms)} atoms to Cartesian coordinates.")

    # Write to the output file
    with open(output_file, 'w') as f:
        f.write(f"{len(converted_atoms)}\n\n")  # Write the number of atoms
        for element, x, y, z in converted_atoms:
            f.write(f"{element} {x:.6f} {y:.6f} {z:.6f}\n")
    
    print(f"Translation complete! Output saved to {output_file}")



In [4]:
# Specify the input and output file paths
input_file = "/Users/austin/Desktop/Projects/BCC_Iron_Yao/4nm_loop_on_111_plane_cif_files/4nm_111111_loop_20nm_sys_structure.cif" 
output_file = "/Users/austin/Desktop/Projects/BCC_Iron_Yao/4nm_loop_on_111_plane_cif_files/4nm_111111_loop_20nm_sys_structure.xyz" 

# Run the function
read_and_translate_new_format(input_file, output_file)

Reading
Read 686279 lines from the input file.
First 10 lines: ['data_generated\n', '_cell_length_a    200.044895\n', '_cell_length_b    200.061307\n', '_cell_length_c    200.045114\n', '_cell_angle_alpha  90.000\n', '_cell_angle_beta   90.000\n', '_cell_angle_gamma  90.000\n', '\n', 'loop_\n', '_atom_site_label\n']
200.044895 200.061307 200.045114
Lattice parameters: a=200.044895, b=200.061307, c=200.045114
Atom site data starts at line 14.
Extracted 686265 atom sites.
Converted 686265 atoms to Cartesian coordinates.
Translation complete! Output saved to /Users/austin/Desktop/Projects/BCC_Iron_Yao/4nm_loop_on_111_plane_cif_files/4nm_111111_loop_20nm_sys_structure.xyz


In [5]:
abtem.config.set({"device": "cpu", "fft": "fftw"})

In [ ]:
# atoms = read('./BCC_iron.cif') * (3, 3, 3)
atoms = read('to_sim_4nm_100_loop.xyz', format='xyz')
a = 200.05
cell = [a, a, a]               
atoms.set_cell(cell)   
atoms.rotate(90, 'x', rotate_cell=True)


In [ ]:
# find n_cells on all the edges,
# make slabs of that size.
# alight the atoms at the edge of the slab with the loop cell
# shift the slab so that edge atoms exactly align with the atoms at the edge of the loop cell
# delete the atoms that now overlap at the edge



In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
abtem.show_atoms(atoms, ax=ax1, title="Beam view", merge=False)
abtem.show_atoms(atoms, ax=ax2, plane="xz", title="Side view", legend=True);

In [ ]:
atoms.cell

In [ ]:
tilt = 4 * np.sqrt(2)
atoms.rotate(tilt, 'x', rotate_cell=False, center='COU')
atoms.rotate(-tilt, 'y', rotate_cell=False, center='COU')

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
abtem.show_atoms(atoms, ax=ax1, title="Beam view", merge=False)
abtem.show_atoms(atoms, ax=ax2, plane="xz", title="Side view", legend=True);

In [ ]:
# Wrap the atoms into the current cell to ensure they're within bounds
atoms.wrap()

# Update the cell to fit tightly around the atoms
#atoms.cell = atoms.get_positions().ptp(axis=0)  # Point-to-point extent along each axis

# Optional: Center the atoms within the new cell
#atoms.center()

# Check the updated cell
print(f"New cell dimensions:\n{atoms.get_cell()}")

In [ ]:
print(f"Number of atoms: {len(atoms)}")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
abtem.show_atoms(atoms, ax=ax1, title="Beam view", merge=False)
abtem.show_atoms(atoms, ax=ax2, plane="xz", title="Side view", legend=True);

In [ ]:
frozen_phonons = abtem.FrozenPhonons(atoms, 8, sigmas=0.1)
potential = abtem.Potential(frozen_phonons, sampling=0.03)

In [ ]:
probe = abtem.Probe(energy=200e3, semiangle_cutoff=4.5, Cs=10e4, defocus=10)
probe.grid.match(potential)

print(f"defocus = {probe.aberrations.defocus} Å")
print(f"FWHM = {probe.profiles().width().compute()} Å")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
probe.show(ax=ax1)
probe.profiles().show(ax=ax2);


### Try PRISM

In [ ]:
s_matrix = abtem.SMatrix(
    potential=potential,
    energy=200e3,
    semiangle_cutoff=4.5,
    interpolation=4
)
s_matrix.build()


### Check the sampling by looking at the entrance and exit probe

In [ ]:
Cs = 8e-6 * 1e10  # 20 micrometers
ctf = abtem.CTF(Cs=Cs, defocus="scherzer", energy=s_matrix.energy)
print(f"defocus = {ctf.defocus} Å")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
s_matrix.dummy_probes(plane="entrance", ctf=ctf).show(ax=ax1, title="entrance", power=1)
s_matrix.dummy_probes(plane="exit", ctf=ctf).show(ax=ax2, title="exit", power=1);

### Set up the scan/ detector/ measurement

In [ ]:
# pixelated detector
detector = abtem.PixelatedDetector(max_angle = 40)

# scan sampling
sampling = abtem.transfer.nyquist_sampling(s_matrix.semiangle_cutoff, s_matrix.energy)
print(f"Sampling: {sampling:.2f} Å^-1")

scan = abtem.GridScan(
    start=(0, 0),
    end=(1,1),
    fractional=True,
    potential=potential,
    sampling=sampling,
)

print(f"Number of probe positions: {len(scan)}")
print(f"Number of plane waves: {len(s_matrix)}")
print(f"Ratio: {len(scan) / len(s_matrix):.1f}")

In [ ]:
measurement= s_matrix.scan(
    scan=scan,
    detectors=detector,
)

measurement.axes_metadata

In [ ]:
measurement.compute()